In [ ]:
import spacy
from nltk.corpus import wordnet as wn

In [ ]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner'])

In [ ]:
all_words = set()
for pos in ['v', 'n', 'a']:
    for synset in wn.all_synsets(pos):
        lemma_names = [x for x in synset.lemma_names() if x.isalpha()]
        all_words.update(lemma_names)
        

In [ ]:
word_vectors = dict()
for word in all_words:
    word_nlp = nlp(word.lower())[0]
    has_vector, vector = word_nlp.has_vector, word_nlp.vector
    if has_vector:
        word_vectors[word] = vector